In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.insert(0, './TetrisEnv')
from TetrisBattle.envs.tetris_env import TetrisSingleEnv, TetrisEnv
from TetrisBattle import *
# %pip install -e ./TetrisEnv
# import TetrisBattle

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from gym.wrappers import GrayScaleObservation
from preprocessing.EensyWeensy import MakeEensyWeensy
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

from gym import ActionWrapper
from gym.spaces import Discrete
class ReduceActions(ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.action_space = Discrete(7)
    def action(self, act):
        if act == 0:
            return 0
        else:
            return act + 1
# from gym import make
# env = make('SinglePTetris-v0')

In [4]:
env = TetrisSingleEnv()
env = GrayScaleObservation(env, keep_dim=True)
env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = ReduceActions(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")
# ob = env.reset()
# plt.imshow(ob, cmap="plasma")

In [5]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
import torch
print(torch.cuda.is_available())
check_env(TetrisSingleEnv())

True


In [6]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [7]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [8]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [9]:
try:
    del model
except NameError:
    pass
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=1e-4, n_steps=128, device="cuda") 
model = PPO.load('./train/cams100000')
model.set_env(env)

Using cuda device
Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.


In [10]:
# model.learn(total_timesteps=1000000, callback=callback)

In [11]:
model = PPO.load('./train/best_model_800000')
env = TetrisSingleEnv()
env = GrayScaleObservation(env, keep_dim=True)
env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = ReduceActions(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")
state = env.reset()

for i in range(0, 10000):
    action, _ = model.predict(state)
    # action = env.action_space.sample()
    # print(action)
    state, reward, done, info = env.step(action)
    env.render()

env.close()